# Extended Step 1 : Zero-to-Nodes (Z2N) and Node-to-Node (N2N) formulations

📜 _If you already know about SDC from the [original paper](https://link.springer.com/content/pdf/10.1023/A:1022338906936.pdf) of Dutt, Greengard and Rokhlin, you may notice that their description is very different from the one given [in Step 4](./04_sdc.ipynb) ..._

Indeed, this tutorial introduced SDC using a **Zero-to-Nodes formulation (Z2N)**, which describes the SDC node updates from the initial step solution (zero) to the node. This approach is identical as the one used to describe Runge-Kutta methods with Butcher tables in the literature.

The SDC authors however used a different formulation, namely the **Node-to-Node formulation (N2N)**, which describes the node update from one node to the next. While both formulations can produce identical SDC schemes, they have some fundamental differences from an implementation perspective, and leads to different generalizations of SDC.

We describe here how to switch from Z2N to N2N (and vice-versa), and how to implement the N2N formulation using `qmat`.

## From Zero-to-Nodes to Node-to-Node

Starting from the Z2N update on the Dahlquist problem :

$$
u^{k+1} - \lambda\Delta{t}Q_\Delta u^{k+1} = u_n + \lambda\Delta{t}(Q-Q_\Delta)u^k,
$$

we can expend it to an update for each node solution $u_{m} \simeq u(t_n+\tau_m\Delta{t}),\; m \in \{1,\dots,M\}$:

$$
u^{k+1}_{m+1} - \lambda\Delta{t}\sum_{j=1}^{m+1}q^\Delta_{m+1,j} u^{k+1}_{j} 
    = u_n 
    + \lambda\Delta{t}\sum_{j=1}^{M}q_{m+1,j}u^{k}_{j}
    - \lambda\Delta{t}\sum_{j=1}^{m+1}q^\Delta_{m+1,j}u^{k}_{j},
$$

where $u_n$ is the initial solution for the time-step (scalar, abusing notation again ...),
and we note $(q^\Delta)_{i,j} := Q_\Delta$ and $(q)_{i,j} := Q$.
Rearranging and regrouping terms, we can write it like this :

$$
u^{k+1}_{m+1} =
    u_n 
    + \lambda\Delta{t}\sum_{j=1}^{m+1}q^\Delta_{m+1,j} (u^{k+1}_{j} - u^{k}_{j}) 
    + \lambda\Delta{t}\sum_{j=1}^{M}q_{m+1,j}u^{k}_{j}.
$$

Now subtracting the update formula for $u^{k+1}_m$ from $u^{k+1}_{m+1}$,
we get the **generic N2N sweep formula** for $m > 0$ (starting from the second node) :

$$
u^{k+1}_{m+1} = u^{k+1}_m
    + \lambda\Delta{t}\sum_{j=1}^{m+1}\left(q^\Delta_{m+1,j} - q^\Delta_{m,j}\right)\left(u^{k+1}_{j} - u^{k}_{j}\right)
    + \lambda\Delta{t}\sum_{j=1}^{M}\left(q_{m+1,j}-q_{m,j}\right)u^{k}_{j},
$$

and for the first node (no subtraction):

$$
u^{k+1}_{1} = u_n 
    + \lambda\Delta{t}q^\Delta_{1,1} (u^{k+1}_{1} - u^{k}_{1}) 
    + \lambda\Delta{t}\sum_{j=1}^{M}q_{1,j}u^{k}_{j}.
$$

> :bell: Note that $q^\Delta_{m,m+1}=0$ because of the lower triangular nature of $Q_\Delta$, so we can add this coefficient
> in the generic N2N sweep formula to simplify it.

Defining $s_{m+1,j} = q_{m+1,j}-q_{m,j} \; \forall m \in \{1, M-1\}$ and $s_{1,j} = q_{1,j}$,
we note $S$ the matrix built with the $(s)_{i,j}$ coefficients,
and write the **generic N2N formula** into matrix formulation :

$$
\begin{pmatrix}
1 \\
-1 & 1 \\
& \ddots & \ddots \\
& & -1 & 1
\end{pmatrix}
(u^{k+1} - u^{k}) 
= \begin{pmatrix}
u0 \\ 0 \\ \vdots \\ 0 
\end{pmatrix}
+ \lambda\Delta{t}S_\Delta (u^{k+1}-u^{k}) + \lambda\Delta{t}S u^k,
$$

where $S_\Delta$ is built from the $Q_\Delta$ matrix the same way as $S$ from $Q$.


**Backward-Euler based sweep**

Consider now one of the original SDC form proposed by Dutt, Greengard and Rokhlin : the Backward-Euler based sweep.
The associated $Q_\Delta$ coefficients are implemented in `qmat`, and considering a simple illustrative node distribution we obtain :

In [1]:
from qmat import genQDeltaCoeffs
genQDeltaCoeffs("BE", form="N2N", nodes=[0.1, 0.3, 0.7, 1.0])

array([[0.1, 0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. ],
       [0. , 0. , 0.4, 0. ],
       [0. , 0. , 0. , 0.3]])

We obtain a triangular matrix with non-zero diagonal (implicit sweep), but with **all non-zero coefficients in each columns being identical**. This implies that for $m \in \{1, M\}$ :

$$
\lambda\Delta{t}\sum_{i=1}^{m}\left(q^\Delta_{m+1,i} - q^\Delta_{m,i}\right)\left(u^{k+1}_{i} - u^{k}_{i}\right) = 0
$$

which simplifies the N2N formula into :

$$
u^{k+1}_{m+1} = u^{k+1}_m
    + \lambda\Delta{t}q^\Delta_{m+1,m+1}\left(u^{k+1}_{m+1} - u^{k}_{m+1}\right)
    + \lambda\Delta{t}\sum_{j=1}^{M}\left(q_{m+1,j}-q_{m,j}\right)u^{k}_{j},
$$

where we can note $u^{k}_0 := u_n\quad\forall k$ so that the formula can be applied on all nodes.
Defining $s_{m+1,j} = q_{m+1,j}-q_{m,j} \quad \forall m \in \{1, M-1\}$ and $s_{1,j} = q_{1,j}$,
we note $S$ the matrix built with the $(s)_{i,j}$ coefficients,
and 
